In [21]:
import csv
import numpy as np
from collections import OrderedDict

In [28]:
filepath = 'data-raw/srs-example.csv'
headers = True

In [29]:
def mean(list):
    return (float(sum(list))/len(list))

class Team(object):
    def __init__(self, name, spreads, opponents):
        self.name = name
        self.spreads = spreads
        self.opponents = opponents

    def set_average_spread(self):
        self.spread = mean(self.spreads)

In [30]:
csvfile = open(filepath, 'r')
gamereader = csv.reader(csvfile, dialect = 'excel')
if headers:
    next(gamereader)

#store all teams in a dict as "team name": team object
# teams = {}
teams = OrderedDict()

#loop through games and construct team objects
for game in gamereader:
    t1 = game[0]
    t2 = game[1]
    # print(t1, t2)
    t1spread = int(game[2]) - int(game[3])
    t2spread = -t1spread

    if t1 in teams:
        teams[t1].spreads.append(t1spread)
        teams[t1].opponents.append(t2)
    else:
        teams[t1] = Team(t1, [t1spread], [t2])

    if t2 in teams:
        teams[t2].spreads.append(t2spread)
        teams[t2].opponents.append(t1)
    else:
        teams[t2] = Team(t2, [t2spread], [t1])

csvfile.close()

#calculate the means
for team in teams.keys():
    teams[team].set_average_spread()

In [31]:
for team in teams.keys():
    print(teams[team].name, teams[team].spread)

Atlanta 4.666666666666667
Boston -11.0
Chicago -9.333333333333334
Dallas -0.6666666666666666
Eugene 9.333333333333334
Fairbanks 7.0


In [32]:
#first matrix with the coefficients of each of the variables
terms = []
#seccond matrix with the constant term (-average spread)
solutions = []
for team in teams.keys():
    #add in a row for each team
    row = []

    #rating = average spread + average opponent rating
    #-> -average spread = -rating + average opponent rating
    #-> -average spread = -rating + 
    #(number of opponents/1) * (opponent 1 rating+opponent 2 rating...)
    #each row of the matrix describes right side equation
    for opp in teams.keys():
        if opp == teams[team].name:
            row.append(-1)
        elif opp in teams[team].opponents:
            row.append(1.0/len(teams[team].opponents))
        else:
            row.append(0)
    terms.append(row)

    #each row of this matrix describes the left side of the above equation
    solutions.append(-teams[team].spread)

In [33]:
# print(terms)

In [20]:
# print(solutions)

[11.0, -4.666666666666667, 0.6666666666666666, 9.333333333333334, -2.0, -14.333333333333334]


In [34]:
array1 = np.array(terms)
array2 = np.array(solutions)
solutions  = np.linalg.solve(array1, array2)
print(array1)
print(array2)
print(solutions)

[[-1.          0.33333333  0.33333333  0.33333333  0.          0.        ]
 [ 0.33333333 -1.          0.          0.          0.33333333  0.33333333]
 [ 0.33333333  0.         -1.          0.33333333  0.33333333  0.        ]
 [ 0.33333333  0.          0.33333333 -1.          0.          0.33333333]
 [ 0.          0.33333333  0.33333333  0.         -1.          0.33333333]
 [ 0.          0.33333333  0.          0.33333333  0.33333333 -1.        ]]
[ -4.66666667  11.           9.33333333   0.66666667  -9.33333333  -7.        ]
[-2.0962963  -8.2962963  -9.22962963 -2.76296296  5.17037037  5.03703704]


In [35]:
rankings = list(zip([teams[team].name for team in teams.keys()], [solution for solution in solutions]))

In [36]:
print(rankings)

[('Atlanta', -2.0962962962962948), ('Boston', -8.2962962962962958), ('Chicago', -9.2296296296296276), ('Dallas', -2.7629629629629622), ('Eugene', 5.1703703703703692), ('Fairbanks', 5.0370370370370363)]
